## 朴素贝叶斯

In [1]:
import numpy as np
import pandas as pd
import jieba
import csv
import sklearn

## 载入数据

In [7]:
data = open('Data/data_new/test.txt','r',encoding='utf-8')
f=open('Data/data_new/4.csv','a',newline='',encoding='utf-8')

csv_writer = csv.writer(f)

m=data.readlines()

csv_writer.writerow(["title","tag"])

for i in m:
    n=i.strip('\n')
    L=n.split('\t')
    # print(L)
    csv_writer.writerow(L)
    
f.close()
data.close()

In [8]:
data=pd.read_csv('Data/data_new/4.csv')
data.head()

,title,tag
0,今年一季度全国纪检监察机关处分省部级干部16人,0
1,习近平：携手迎接挑战 合作开创未来,0
2,瞭望丨引领人类海洋文明发展方向,0
3,广东省老区湾区同携手 接续奋进新时代,0
4,关注全民健身：健身场地少、消费高等难题怎么破？,0


## 划分训练集测试集

In [9]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data['title'], data['tag'], random_state=1)



In [10]:
print(x_test.shape[0])
print(type(y_test))

8735
<class 'pandas.core.series.Series'>


## 分词

In [11]:
def fenci(train_data):
    words_df = train_data.apply(lambda x:' '.join(jieba.cut(x)))
    return words_df

x_train_fenci = fenci(x_train)
x_train_fenci[:5]


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\32852\AppData\Local\Temp\jieba.cache
Loading model cost 0.635 seconds.
Prefix dict has been built successfully.


33275                                  易经 — （ 月 象图 ）
25108               王思聪 现身 防疫站 打 疫苗 ， 王 可可 看来 要 失宠 了
30563    英格兰 的 新 金童   能 从 普通 球星 上升 到 梅西 这样 的 超级 巨星 吗
18379              无证 驾驶 ， 造成 人身 伤亡 ， 交强险 是否 也 不 赔 ？
26937          为国争光 ！ 中国乒乓球队 凯旋归来   球迷 热情 迎接   大满贯 ！
Name: title, dtype: object

## 停词

In [12]:
infile = open("hit_stopwords.txt",encoding='utf-8')
stopwords_lst = infile.readlines()
stopwords = [x.strip() for x in stopwords_lst]

## 向量化

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words=stopwords, max_features=5000)
vectorizer.fit(x_train_fenci)

D:\Software\Anaconda\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


CountVectorizer(max_features=5000,
                stop_words=['———', '》），', '）÷（１－', '”，', '）、', '＝（', ':', '→',
                            '℃', '&', '*', '一一', '~~~~', '’', '.', '『', '.一',
                            './', '--', '』', '＝″', '【', '［＊］', '｝＞', '［⑤］］',
                            '［①Ｄ］', 'ｃ］', 'ｎｇ昉', '＊', '//', ...])

In [14]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
#模型训练
classifier.fit(vectorizer.transform(x_train_fenci), y_train)
#使用训练好的模型进行预测
classifier.score(vectorizer.transform(fenci(x_test)), y_test)

0.8022896393817973

## 预测

In [15]:
from sklearn.metrics import classification_report
predict=classifier.predict(vectorizer.transform(fenci(x_test)))
print("每个类别的精确率与召回率",classification_report(predict, y_test))

每个类别的精确率与召回率               precision    recall  f1-score   support

           0       0.58      0.75      0.66       444
           1       0.74      0.79      0.76       864
           2       0.77      0.69      0.73       645
           3       0.82      0.79      0.81       940
           4       0.86      0.85      0.86      1155
           5       0.86      0.85      0.86      1219
           6       0.84      0.71      0.77      1537
           7       0.88      0.88      0.88      1229
           8       0.69      0.87      0.77       702

    accuracy                           0.80      8735
   macro avg       0.78      0.80      0.79      8735
weighted avg       0.81      0.80      0.80      8735



In [16]:
f=open('test_5.txt','r',encoding='utf-8')

L=[]
m=f.readlines()
for i in m:
    n=i.strip('\n')
    x=n.split('\t')
    L.append(x[1])

data_to_predict=pd.Series(L)
# data_to_predict=np.matrix(data_to_predict)

In [17]:
print(data_to_predict.shape)

(603,)


In [18]:
predict=classifier.predict(vectorizer.transform(fenci(data_to_predict)))

In [19]:
print(predict)

[5 4 1 6 3 6 6 3 6 6 7 3 4 7 3 0 5 5 1 5 0 4 7 8 4 5 3 6 3 7 1 2 0 4 3 5 4
 6 3 3 5 3 1 4 5 7 4 0 1 3 5 0 5 7 7 8 0 2 2 0 3 6 3 3 3 0 6 6 2 0 6 3 1 4
 0 8 4 5 1 3 0 5 4 3 6 0 4 3 3 0 2 4 5 6 5 0 3 7 3 5 5 5 7 4 4 1 3 7 5 0 4
 6 7 3 3 5 4 3 6 6 0 1 5 1 7 7 0 6 5 1 1 4 4 1 3 5 8 0 0 2 7 3 2 1 5 4 0 1
 1 3 4 1 6 3 0 1 0 7 0 1 6 6 3 5 2 0 0 6 4 1 4 0 6 6 7 5 4 3 2 3 4 1 4 3 1
 6 6 6 5 4 3 6 6 5 7 1 2 4 8 0 1 3 7 3 1 0 8 5 8 2 7 5 2 6 4 7 0 4 4 7 8 2
 5 2 3 2 4 4 3 3 6 0 5 1 0 6 6 6 3 0 4 6 0 4 2 4 1 0 2 3 7 2 3 3 7 6 7 7 6
 0 3 5 3 0 8 4 3 7 7 8 3 4 1 5 6 2 4 6 7 3 2 4 4 5 1 7 7 3 2 2 8 4 6 7 3 0
 4 8 5 1 3 8 0 0 8 5 0 8 5 8 7 8 2 4 0 2 6 0 0 4 6 4 4 3 5 6 6 4 0 0 6 0 2
 8 3 0 0 7 2 4 5 6 2 4 6 1 4 7 6 2 0 7 7 4 6 0 4 3 1 2 0 3 4 8 0 5 8 1 5 6
 4 3 6 5 1 2 5 1 6 5 2 4 3 5 4 1 0 0 6 7 4 5 1 6 5 4 6 4 6 1 3 0 3 7 7 3 5
 4 4 3 5 1 5 5 0 0 5 7 0 3 7 3 7 1 0 7 4 5 3 0 5 6 7 5 7 0 5 2 2 1 5 8 0 4
 4 2 5 3 4 7 3 5 4 7 4 5 3 6 3 1 8 7 3 2 4 3 3 3 6 3 1 4 7 0 5 6 5 1 1 4 2
 2 3 7 0 6 7 2 0 3 3 4 0 

## TF-IDF预处理

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
#使用tf-idf把文本转为向量
tv = TfidfVectorizer(stop_words=stopwords, max_features=5000, lowercase = False)
tv.fit(x_train_fenci)
#模型训练
classifier.fit(tv.transform(fenci(x_train)), y_train)
#利用训练好的模型测试
classifier.score(tv.transform(fenci(x_test)), y_test)


D:\Software\Anaconda\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①Ａ', '①Ｂ', '①Ｃ', '①Ｄ', '①Ｅ', '①ａ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②Ｂ', '②Ｇ', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③Ｆ', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


0.7978248425872925

## N-gram 模型

In [21]:
tv_2gram = TfidfVectorizer(stop_words=stopwords, max_features=5000, ngram_range=(1,2),lowercase = False)
tv_2gram.fit(x_train_fenci)
#训练模型
clf_2gram = MultinomialNB()
clf_2gram.fit(tv_2gram.transform(fenci(x_train)), y_train)
#预测
clf_2gram.score(tv_2gram.transform(fenci(x_test)), y_test)

D:\Software\Anaconda\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①Ａ', '①Ｂ', '①Ｃ', '①Ｄ', '①Ｅ', '①ａ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②Ｂ', '②Ｇ', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③Ｆ', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


0.7935890097309674